In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
# Data processing
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Model and performance
import tensorflow as tf
from tensorflow.python.keras import layers, losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

RANDOM_STATE_SEED = 12

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Kaggle/02-23-2018.csv')
df = df.drop(['Timestamp'], axis=1)
df.replace([-np.inf, np.inf], np.nan, inplace = True)
df.dropna(inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1042867 entries, 0 to 1048574
Data columns (total 79 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1042867 non-null  int64  
 1   Protocol           1042867 non-null  int64  
 2   Flow Duration      1042867 non-null  int64  
 3   Tot Fwd Pkts       1042867 non-null  int64  
 4   Tot Bwd Pkts       1042867 non-null  int64  
 5   TotLen Fwd Pkts    1042867 non-null  int64  
 6   TotLen Bwd Pkts    1042867 non-null  int64  
 7   Fwd Pkt Len Max    1042867 non-null  int64  
 8   Fwd Pkt Len Min    1042867 non-null  int64  
 9   Fwd Pkt Len Mean   1042867 non-null  float64
 10  Fwd Pkt Len Std    1042867 non-null  float64
 11  Bwd Pkt Len Max    1042867 non-null  int64  
 12  Bwd Pkt Len Min    1042867 non-null  int64  
 13  Bwd Pkt Len Mean   1042867 non-null  float64
 14  Bwd Pkt Len Std    1042867 non-null  float64
 15  Flow Byts/s        1042867 non-n

In [ ]:
df['Label'].value_counts()

Benign              1042301
Brute Force -Web        362
Brute Force -XSS        151
SQL Injection            53
Name: Label, dtype: int64

In [ ]:
df.replace(to_replace=["Brute Force -Web", "Brute Force -XSS", "SQL Injection"], value="Malicious", inplace=True)
df.shape

(1042867, 79)

In [ ]:
df.replace(to_replace = 'Benign', value = 0, inplace=True)
df.replace(to_replace = 'Malicious', value = 1, inplace=True)

In [ ]:
# Initialize the scaler
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), columns=df.columns)

In [ ]:
from google.colab import files
df.to_csv('dataset7.csv', encoding = 'utf-8-sig')
files.download('dataset7.csv')

In [ ]:
dfmal = df.loc[df['Label'] == 1]
dfmal

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
339,0.001221,0.352941,4.851640e-02,0.000034,0.000008,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.0
340,0.001221,0.352941,4.611041e-01,0.002634,0.000869,0.034406,0.000517,0.097998,0.000000,0.195363,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.0
341,0.001221,0.352941,1.466667e-06,0.000017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.0
342,0.000000,0.000000,9.386749e-01,0.000034,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.469370,7.507864e-07,0.469370,0.469370,1.0
343,0.007630,1.000000,7.456625e-01,0.000086,0.000000,0.001896,0.000000,0.075850,0.345066,0.276311,...,0.166667,0.035347,0.0,0.035347,0.035347,0.178094,2.134862e-01,0.349943,0.060008,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588,0.001221,0.352941,3.750001e-07,0.000017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.0
1998,0.001221,0.352941,2.300001e-06,0.000017,0.000008,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.0
2115,0.001221,0.352941,2.750001e-06,0.000017,0.000008,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.0
2216,0.001221,0.352941,2.600001e-06,0.000017,0.000008,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.0


In [ ]:
dfben = df.loc[df['Label'] == 0]
dfben

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.000336,0.352941,0.012772,0.000172,0.000089,0.000745,1.259270e-05,0.098301,0.000000,0.059231,...,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.007630,1.000000,0.979782,0.000034,0.000000,0.000948,0.000000e+00,0.075850,0.345066,0.276311,...,0.166667,0.000000,0.0,0.000000,0.000000,0.489925,0.331849,0.629902,0.349948,0.0
2,0.007630,1.000000,0.979782,0.000034,0.000000,0.000948,0.000000e+00,0.075850,0.345066,0.276311,...,0.166667,0.000000,0.0,0.000000,0.000000,0.489925,0.331849,0.629902,0.349948,0.0
3,0.000336,0.352941,0.014545,0.000172,0.000089,0.000745,1.259270e-05,0.098301,0.000000,0.059231,...,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.007630,1.000000,0.745696,0.000086,0.000000,0.001896,0.000000e+00,0.075850,0.345066,0.276311,...,0.166667,0.035348,0.0,0.035348,0.035348,0.178102,0.213485,0.349938,0.060008,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042862,0.825770,0.352941,0.716604,0.000017,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.716654,0.000000,0.716654,0.716654,0.0
1042863,0.517502,0.352941,0.000001,0.000017,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1042864,0.000809,1.000000,0.000659,0.000000,0.000008,0.000026,6.267571e-07,0.006220,0.028295,0.022658,...,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1042865,0.051714,0.352941,0.083782,0.000172,0.000097,0.000835,1.061010e-05,0.102700,0.000000,0.066365,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [ ]:
df.loc[df['Label'] == 0].shape

(1042301, 79)

In [ ]:
dfmal.shape

(566, 79)

In [ ]:
df.loc[df['Label'] == 1].shape

(566, 79)

In [ ]:
dfben.drop('Label', axis=1, inplace=True)
dfmal.drop('Label', axis=1, inplace=True)
dfben = dfben.values
dfmal = dfmal.values

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
train, test = train_test_split(dfben, test_size=0.25, random_state=42)

In [ ]:
train.shape

(781725, 78)

In [ ]:
from sklearn import metrics
import numpy as np
import pandas as pd
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(78, input_dim=train.shape[1], activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(train.shape[1]))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 78)                6162      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2528      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 16)                1

In [ ]:
model.fit(train, train, verbose=1, epochs=10)

Epoch 1/10
24429/24429 [==============================] - 69s 3ms/step - loss: 0.0162
Epoch 2/10
24429/24429 [==============================] - 66s 3ms/step - loss: 0.0065
Epoch 3/10
24429/24429 [==============================] - 68s 3ms/step - loss: 0.0061
Epoch 4/10
24429/24429 [==============================] - 65s 3ms/step - loss: 0.0059
Epoch 5/10
24429/24429 [==============================] - 64s 3ms/step - loss: 0.0058
Epoch 6/10
24429/24429 [==============================] - 67s 3ms/step - loss: 0.0057
Epoch 7/10
24429/24429 [==============================] - 74s 3ms/step - loss: 0.0056
Epoch 8/10
24429/24429 [==============================] - 62s 3ms/step - loss: 0.0056
Epoch 9/10
24429/24429 [==============================] - 64s 3ms/step - loss: 0.0056
Epoch 10/10
24429/24429 [==============================] - 64s 3ms/step - loss: 0.0056


In [ ]:
pred = model.predict(test)
score1 = np.sqrt(metrics.mean_squared_error(pred, test))

In [ ]:
pred

array([[-3.7223585e-03,  3.6192542e-01,  9.7118390e-01, ...,
         2.4444478e-02,  4.6724385e-01,  4.4814807e-01],
       [-9.4994344e-03,  3.2956025e-01,  1.9506868e-02, ...,
         6.7871436e-04,  9.8232701e-03,  8.3082803e-03],
       [ 4.4115940e-01,  3.6720234e-01,  7.2715509e-01, ...,
         2.0357855e-03,  7.0505273e-01,  7.0644921e-01],
       ...,
       [ 1.7642021e-01,  6.9403112e-01,  2.2548249e-03, ...,
         8.2846172e-04,  7.6558180e-03,  6.6777319e-03],
       [ 4.4648509e-02,  3.6210507e-01,  3.4305744e-02, ...,
        -8.6979382e-04,  4.5854189e-03,  4.5263991e-03],
       [ 1.7642018e-01,  6.9403118e-01,  2.2548372e-03, ...,
         8.2846358e-04,  7.6558292e-03,  6.6777430e-03]], dtype=float32)

In [ ]:
pred = model.predict(test)
score2 = np.sqrt(metrics.mean_squared_error(pred, test))

In [ ]:
pred = model.predict(dfmal)
score3 = np.sqrt(metrics.mean_squared_error(pred, dfmal))

In [ ]:
print(f"Train Dataset Score (RMSE): {score1}".format(score1))
print(f"Test Dataset Score (RMSE): {score2}")
print(f"Malicious Dataset Score (RMSE): {score3}")

Train Dataset Score (RMSE): 0.07402736134553389
Test Dataset Score (RMSE): 0.07402736134553389
Malicious Dataset Score (RMSE): 0.14505965504242502
